In [ ]:
!cd /content
!rm tomidius -rf
!git clone https://github.com/FIAV1/tomidius.git

Cloning into 'tomidius'...
remote: Enumerating objects: 1108, done.
remote: Counting objects: 100% (1108/1108), done.
remote: Compressing objects: 100% (1105/1105), done.
remote: Total 1108 (delta 3), reused 1104 (delta 2), pack-reused 0
Receiving objects: 100% (1108/1108), 2.68 MiB | 14.77 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Filtering content: 100% (1087/1087), 29.09 MiB | 2.13 MiB/s, done.


In [ ]:
import torch
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
class Red(nn.Module):
  def _init_(self, *args, **kwargs):
    super()._init_(*args, **kwargs)
    # Capas convolucionales
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
    self.conv4 = nn.Conv2d(128, 256, kernel_size=3)
    self.pool = nn.MaxPool2d(kernel_size=2)
    self.flatten = nn.Flatten()
    self.lin1 = nn.Linear(4096, 2048)
    self.lin2 = nn.Linear(2048, 256)
    self.lin3 = nn.Linear(256, 1)
    self.relu = nn.ReLU()
  def forward(self, x, temp: float):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.pool(self.relu(self.conv3(x)))
    x = self.pool(self.relu(self.conv4(x)))
    x = self.flatten(x)
    x = self.relu(self.lin1(x))
    x = self.relu(self.lin2(x))
    x = self.relu(self.lin3(x))
    x = x / temp
    return x
red = Red().cuda()
ts = transforms.Compose(
    [
        transforms.RandomRotation(180),
        transforms.RandomHorizontalFlip(),
        transforms.Resize((100, 100)),
        transforms.ToTensor()
    ]
)
ds = ImageFolder(root="/content/tomidius/dataset/train", transform=ts)
loader = DataLoader(dataset=ds, batch_size=64, shuffle=True)
loss = nn.BCELoss()
opt = optim.Adam(red.parameters(), lr=0.001)
for t in tqdm(range(15)):
  for imagenes, etiquetas in loader:
    opt.zero_grad()
    salidas = red.forward(imagenes.cuda(), 1).cuda()
    error = loss(salidas, etiquetas.cuda().unsqueeze(1).float())
    error.backward()
    opt.step()
    precision = _, pred = torch.max(salidas, 1)
    correctas = torch.sum(salidas == etiquetas.cuda()).item()
    totales = etiquetas.shape[0]
    precision = correctas/totales
  print("Precision:", precision)
torch.save(red.load_state_dict(), "modelo.pth")

ValueError: optimizer got an empty parameter list